In [10]:
with open("import_modules.py") as f:
    exec(f.read())

from graph import GraphViz

All imported


In [53]:
from sklearn.ensemble import RandomForestClassifier
from joblib import load
from sentence_transformers import SentenceTransformer
import numpy as np

def prepreocessing(final):
    final=final.copy()
    model = SentenceTransformer('all-MiniLM-L6-v2')
    final.loc[:, 'arg1'] = final.loc[:, 'arg1'].apply(lambda x: model.encode(x))
    final.loc[:, 'rel'] = final.loc[:, 'rel'].apply(lambda x: model.encode(x))
    final.loc[:, 'arg2'] = final.loc[:, 'arg2'].apply(lambda x: model.encode(x))
    embeddings_arg1 = np.vstack(final.loc[:, 'arg1'].values)
    embeddings_rel = np.vstack(final.loc[:, 'rel'].values)
    embeddings_arg2 = np.vstack(final.loc[:, 'arg2'].values)
    # final.loc[:, 'negated']=final.loc[:, 'negated'].apply(lambda x: 1 if x==True else 0)
    # final.loc[:, 'passive']=final.loc[:, 'passive'].apply(lambda x: 1 if x==True else 0)
    # negated = np.expand_dims(final.loc[:, 'negated'].values, axis=1)
    # passive = np.expand_dims(final.loc[:, 'passive'].values, axis=1)
    confidence= final.loc[:, ['confidence']].values
    # X = np.hstack([embeddings_arg1, embeddings_rel, embeddings_arg2, negated, passive, confidence])
    X = np.hstack([embeddings_arg1, embeddings_rel, embeddings_arg2, confidence])
    return X


def apply_model(X):
    model = load('model_file.joblib')
    return model.predict(X)



In [54]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(ngram_range=(1,2))

def merge_concepts(df):
    #compute similarity matrix between subjects
    nodes = df['arg1'].unique()
    similarity_matrix = cosine_similarity(vectorizer.fit_transform(nodes))
    
    #create clusters of similar subjects
    set_list = []
    for i in range(len(nodes)):
        s = set([nodes[j] for j in range(len(nodes)) if i != j if similarity_matrix[i][j] > 0.3])
        s.add(nodes[i])
        set_list.append(s)
    for i in range(len(set_list)):
        for j in range(i+1, len(set_list)):
            if set_list[i] & set_list[j]:
                set_list[i] = set_list[i] | set_list[j]
                set_list[j] = set()
    set_list = [s for s in set_list if s]
    
    #create dictionary to substitute subjects with the simpliest one in the cluster
    dict = {}
    for s in set_list:
        simplest_concept = min(s, key=lambda x: len(x))
        for concept in s:
            dict[concept] = simplest_concept
    
    #substitute subjects in dataframe
    df.arg1= df.arg1.map(dict)
    return df

In [55]:
#Remove duplicates, subphrases, and similar phrases
def filter_rows(df):
    seen = set()
    for i, row in df.iterrows():
        phrase = row['arg1'] + ' ' + row['rel'] + ' ' + row['arg2']
        if phrase in seen:
            df = df.drop(i)
        else:
            seen.add(phrase)
    df = df.reset_index(drop=True)
    for l1 in list(seen):
        for l2 in list(seen):
            if l1 != l2 and l1 in l2:
                try:
                    seen.remove(l1)
                except KeyError:
                    continue
    df = df[(df['arg1'] + ' ' + df['rel'] + ' ' + df['arg2']).isin(seen)].reset_index(drop=True)
    return df

def remove_similar_phrases(df):
    rem_ind = set()
    df['phrase'] = df['arg1'] + ' ' + df['rel'] + ' ' + df['arg2']
    sim = cosine_similarity(vectorizer.fit_transform(df['phrase']))
    for i in range(len(sim)):
        for j in range(i+1, len(sim)):
            if sim[i][j] > 0.30:
                if len((df['phrase'].unique()[i]).split(' ')) > len((df['phrase'].unique()[j]).split(' ')):
                    rem_ind.add(j)
                else:
                    rem_ind.add(i)
    return df.drop(rem_ind).reset_index(drop = True).drop('phrase', axis = 1)

In [56]:
DICT_COMM = {}
def concat_objects(relation):
    object_list = relation['arg2'].tolist()
    max_conf = np.max(relation['confidence'])
    common = ''
    first = object_list[0].split(' ')
    for i in range(len(first)):
        if all(obj.startswith(common) for obj in object_list):
            if common == '':
                common += first[i] 
            elif i < len(first) - 1:
                common += ' ' + first[i]
        else:
            common = ' '.join(common.split(' ')[:-1])
            break
    common = common.rstrip()  
    # remove trailing spaces
    if common == '':
        return max_conf, ', '.join(object_list)
    objects = [obj[len(common):].lstrip() for obj in object_list]
    if all(obj == '' for obj in objects):
        return max_conf, (common).rstrip()
    if len(objects) == 1:
        return max_conf, (common + ' ' + objects[0]).rstrip()
    DICT_COMM[common] = objects
    return max_conf, (common + ' ' + ', '.join(objects[:-1]) + ' and ' + objects[-1]).rstrip()

#Concatenate subjects with 'and' where there is the same object and relation
def concat_subjects(relation):
    subject_list = relation['arg1'].tolist()
    max_conf = np.max(relation['confidence'])
    return max_conf, (' and '.join(subject_list)).rstrip()

def concat_concepts(df):
    result = df.groupby(['arg1', 'rel']).apply(concat_objects).reset_index()
    result['confidence'], result['arg2'] = zip(*result[0])
    result = result.drop(columns = 0).sort_values(by='confidence', ascending=False).reset_index(drop=True)
    result = result.groupby(['arg2', 'rel']).apply(concat_subjects).reset_index()
    result['confidence'], result['arg1'] = zip(*result[0])
    result = result.drop(columns = 0).sort_values(by='confidence', ascending=False).reset_index(drop=True)
    result = result[['confidence', 'arg1', 'rel', 'arg2']]
    return result


In [57]:
# def cmap_pipeline(df, min_conf_level, index):
#     DICT_COMM.clear()
#     df = df[df['confidence'] >= df["confidence"].quantile(min_conf_level)]
#     df = merge_concepts(df)
#     df = filter_rows(df)
#     df = concat_concepts(df)
#     df = df[df['arg2'].apply(lambda x: len(x.split(' ')) > 1)].reset_index(drop = True)
#     df = df[df['rel'].apply(lambda x: len(x.split(' ')) < 6)].reset_index(drop = True)
#     df = remove_similar_phrases(df)
#     G = GraphViz(df)
#     G.graph.write_png(f'example_graph_{index}.png')

In [70]:
def cmap_pipeline(df, min_conf_level, index):
    DICT_COMM.clear()
    df = df[df['confidence'] >= df["confidence"].quantile(min_conf_level)]
    input=prepreocessing(df)
    res=apply_model(input)
    df = df.loc[res==1]
    df = merge_concepts(df)
    df = filter_rows(df)
    df = concat_concepts(df)
    df = df[df['arg2'].apply(lambda x: len(x.split(' ')) > 1)].reset_index(drop = True)
    df = df[df['rel'].apply(lambda x: len(x.split(' ')) < 6)].reset_index(drop = True)
    df = remove_similar_phrases(df)
    G = GraphViz(df)
    G.graph.write_png(f'example_graph_{index}.png')

In [71]:
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)

In [72]:
#The first one works, the second one doesn't
cmap_pipeline(data['Reykjavík Summit'], 0.75, 10)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
#The first one works, the second one doesn't
cmap_pipeline(data['Palestinian Expatriates'], 0.82, 1)

In [22]:
# Sembra ci sia un problema con qualche carattere speciale nel testo, ma è solo nel salvare l'immagine che da errore (write_png)
# Ho provato a sostituire graphviz and pydot con networkx + graphviz_layout (lo portavano come soluzione del problema su stackoverflow), 
# ed è dove ho avuto l'altro errore. 
# THANK YOU! ;) 